# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""../../../../data/SampleHDS.jso…","""15280140525699483197""",1.1679e19,"""4447927952478201279""","""722350965927575737""","""18399007478818039979""",false
"""../../../../data/SampleHDS.jso…","""7923828690523560420""",1.3042e19,"""1574730857401546490""","""722350965927575737""","""8768595764755654316""",true
"""../../../../data/SampleHDS.jso…","""9584351597083144920""",8.6076e18,"""15430728959867320320""","""7974998078145320706""","""18399007478818039979""",false
"""../../../../data/SampleHDS.jso…","""5777702472433487312""",5.5894e18,"""1574730857401546490""","""4501274940539784547""","""18399007478818039979""",false
"""../../../../data/SampleHDS.jso…","""17670951671253792085""",6.4289e18,"""4447927952478201279""","""4501274940539784547""","""9532974405906098897""",true
"""../../../../data/SampleHDS.jso…","""7037966979085038478""",1.0894e19,"""15430728959867320320""","""11525710105334284320""",null,false
"""../../../../data/SampleHDS.jso…","""5160512085252316610""",1.2269e19,"""15430728959867320320""","""4501274940539784547""","""9532974405906098897""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_City': 'PREDICTOR_1',
 'Customer_CLV': 'PREDICTOR_2',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""../../../../data/SampleHDS.jso…","""14426643651707395722""",6.9703e18,"""4305299936953042933""","""FirstMortgage30yr""","""5228241715583691007""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""17792490247183375123""",1.4791e19,"""4940222697893725609""","""FirstMortgage30yr""","""16635235941615004799""","""Accepted"""
"""../../../../data/SampleHDS.jso…","""16727995605934336514""",1.6734e19,"""3294867186942065027""","""MoneyMarketSavingsAccount""","""5228241715583691007""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""8882356290101160266""",1.8173e19,"""4940222697893725609""","""BasicChecking""","""5228241715583691007""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""15769118079577196520""",8.4167e18,"""4305299936953042933""","""BasicChecking""","""1010759931695078522""","""Accepted"""
"""../../../../data/SampleHDS.jso…","""12003963665933592555""",7.5866e17,"""3294867186942065027""","""UPlusFinPersonal""",null,"""Rejected"""
"""../../../../data/SampleHDS.jso…","""12970203666497021624""",1.6983e19,"""3294867186942065027""","""BasicChecking""","""1010759931695078522""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""../../../../data/SampleHDS.jso…","""14741867173223555278""",1.8367e19,"""4102406981420599499""","""FirstMortgage30yr""","""5536417524520881704""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""11988065933842333079""",1.6944e19,"""13890548553066681752""","""FirstMortgage30yr""","""5120512997531117083""","""Accepted"""
"""../../../../data/SampleHDS.jso…","""1234975428816348605""",1.6770e19,"""9018326325993387784""","""MoneyMarketSavingsAccount""","""5536417524520881704""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""3021166077484673924""",1.4037e19,"""13890548553066681752""","""BasicChecking""","""5536417524520881704""","""Rejected"""
"""../../../../data/SampleHDS.jso…","""10586804509575342053""",3.3235e17,"""4102406981420599499""","""BasicChecking""","""10662157700059279739""","""Accepted"""
"""../../../../data/SampleHDS.jso…","""1895428620075581790""",7.0462e18,"""9018326325993387784""","""UPlusFinPersonal""",null,"""Rejected"""
"""../../../../data/SampleHDS.jso…","""1191601673004097561""",1.7025e19,"""9018326325993387784""","""BasicChecking""","""10662157700059279739""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_City=PREDICTOR_1
Customer_CLV=PREDICTOR_2
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""14787281134004715964""",1.0424e18,"""8056798468792061053""","""FirstMortgage30yr""","""738857298724152447""","""Rejected"""
"""9833684049645099581""",3.7149e18,"""7488543892175255101""","""FirstMortgage30yr""","""6923149415607301891""","""Accepted"""
"""16672864538958714371""",4.5817e18,"""13389814456435929199""","""MoneyMarketSavingsAccount""","""738857298724152447""","""Rejected"""
"""9414383597374112501""",7.6064e18,"""7488543892175255101""","""BasicChecking""","""738857298724152447""","""Rejected"""
"""16173500020146359275""",1.2998e19,"""8056798468792061053""","""BasicChecking""","""73703120556712166""","""Accepted"""
"""16875030809259747413""",3.6625e18,"""13389814456435929199""","""UPlusFinPersonal""",null,"""Rejected"""
"""7852092479464521087""",1.4138e19,"""13389814456435929199""","""BasicChecking""","""73703120556712166""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'